<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import pandas as pd
import numpy as np
import os as os
import matplotlib.pyplot as plt
import statistics as stats
import seaborn as sns 
import bokeh as bk

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [2]:
# Create an output folder.
# os.mkdir('output')
# Create a files variable that contains all of our data files.
files = os.listdir('data')

In [3]:
files = ['data/Jan 1.xlsx', 'data/Jan 2.xlsx', 'data/Jan 3.xlsx','data/Jan 4.xlsx','data/Jan 5.xlsx','data/Jan 6.xlsx','data/Jan 7.xlsx','data/Jan 8.xlsx','data/Jan 9.xlsx','data/Jan 10.xlsx', 'data/Jan 11.xlsx', 'data/Jan 12.xlsx','data/Jan 13.xlsx', 'data/Jan 14.xlsx', 'data/Jan 15.xlsx' , 'data/Jan 17.xlsx', 'data/Jan 18.xlsx', 'data/Jan 19.xlsx', 'data/Jan 20.xlsx', 'data/Jan 21.xlsx','data/Jan 22.xlsx','data/Jan 23.xlsx','data/Jan 24.xlsx','data/Jan 25.xlsx','data/Jan 26.xlsx','data/Jan 27.xlsx','data/Jan 28.xlsx','data/Jan 29.xlsx','data/Jan 30.xlsx','data/Jan 31.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [4]:
# Read in data from your city from January 1st.

data_Jan_df = pd.read_excel('data/Jan 1.xlsx', sheet_name="New York")

In [5]:
data_Jan_df.head()

,prodcode,price_eu,weight_kg,quantity
0,4159,2.048141,8.812961,138
1,4062,2.728485,9.331372,184
2,4404,2.182498,8.937798,491
3,4412,2.342694,7.067682,218
4,4131,1.802839,9.551503,391


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [6]:
data_Jan_df['price_us']=data_Jan_df['price_eu'] * 1.1

In [7]:
data_Jan_df['weight_lb']=data_Jan_df['weight_kg'] * 2.2

In [8]:
data_Jan_df.head()

,prodcode,price_eu,weight_kg,quantity,price_us,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515
1,4062,2.728485,9.331372,184,3.001334,20.529018
2,4404,2.182498,8.937798,491,2.400748,19.663156
3,4412,2.342694,7.067682,218,2.576964,15.548899
4,4131,1.802839,9.551503,391,1.983123,21.013306


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [9]:
plu = pd.read_csv('plu-codes.csv')

In [10]:
data_Jan_df=pd.merge(data_Jan_df, plu, how='left', left_on='prodcode', right_on='plu_code')

In [11]:
data_Jan_df.head()

,prodcode,price_eu,weight_kg,quantity,price_us,weight_lb,product,plu_code
0,4159,2.048141,8.812961,138,2.252955,19.388515,Onion,4159
1,4062,2.728485,9.331372,184,3.001334,20.529018,Cucumber,4062
2,4404,2.182498,8.937798,491,2.400748,19.663156,Peach,4404
3,4412,2.342694,7.067682,218,2.576964,15.548899,Pear,4412
4,4131,1.802839,9.551503,391,1.983123,21.013306,Apple (Fuji),4131


### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [12]:
data_Jan_df.drop(['price_eu', 'weight_kg', 'plu_code'], axis = 1, inplace=True)

In [13]:
data_Jan_df.head()

,prodcode,quantity,price_us,weight_lb,product
0,4159,138,2.252955,19.388515,Onion
1,4062,184,3.001334,20.529018,Cucumber
2,4404,491,2.400748,19.663156,Peach
3,4412,218,2.576964,15.548899,Pear
4,4131,391,1.983123,21.013306,Apple (Fuji)


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [14]:
#establishing a Date column into the data frame; format source: https://datascience.stackexchange.com/questions/48335/how-to-add-date-column-in-python-pandas-dataframe
data_Jan_df['date'] = pd.date_range(start='01/01/2020', periods=len(data_Jan_df), freq='D')

In [15]:
data_Jan_df.head()

,prodcode,quantity,price_us,weight_lb,product,date
0,4159,138,2.252955,19.388515,Onion,2020-01-01
1,4062,184,3.001334,20.529018,Cucumber,2020-01-02
2,4404,491,2.400748,19.663156,Peach,2020-01-03
3,4412,218,2.576964,15.548899,Pear,2020-01-04
4,4131,391,1.983123,21.013306,Apple (Fuji),2020-01-05


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [16]:
def process_data(file, city):
    df = pd.read_excel(file, sheet_name=city)
    df['price_usd'], df['weight_lb'] = df['price_eu'] * 1.1, df['weight_kg'] * 2.2
    plu= pd.read_csv('plu-codes.csv')
    df=df.merge(plu, how= 'left', left_on='prodcode', right_on='plu_code')
    df.drop(['price_eu', 'weight_kg', 'plu_code'], axis=1, inplace=True)
    df['date']=f'{file[5:-5]}'
    return df
process_data('data/Jan 1.xlsx','Boston')

,prodcode,quantity,price_usd,weight_lb,product,date
0,4134,498,2.153334,17.287309,Apple (Gala),Jan 1
1,4012,384,1.003821,12.642420,Orange,Jan 1
2,4323,431,0.814828,19.247589,Strawberries,Jan 1
3,4078,457,1.728686,15.536389,Corn,Jan 1
4,4323,316,1.528344,2.458212,Strawberries,Jan 1
...,...,...,...,...,...,...
172,4131,309,0.864199,7.000650,Apple (Fuji),Jan 1
173,4958,144,1.946128,3.702045,Lemon,Jan 1
174,4272,455,2.057089,9.947643,Grapes,Jan 1
175,3302,482,0.936777,2.459543,Apricot,Jan 1


In [17]:
# process_data('data/Jan 2.xlsx', 'Boston').head(11)

Test your function out on a new file and city!

In [18]:
[process_data('data/Jan 1.xlsx', 'Boston') for file in files]

[     prodcode  quantity  price_usd  weight_lb       product   date
 0        4134       498   2.153334  17.287309  Apple (Gala)  Jan 1
 1        4012       384   1.003821  12.642420        Orange  Jan 1
 2        4323       431   0.814828  19.247589  Strawberries  Jan 1
 3        4078       457   1.728686  15.536389          Corn  Jan 1
 4        4323       316   1.528344   2.458212  Strawberries  Jan 1
 ..        ...       ...        ...        ...           ...    ...
 172      4131       309   0.864199   7.000650  Apple (Fuji)  Jan 1
 173      4958       144   1.946128   3.702045         Lemon  Jan 1
 174      4272       455   2.057089   9.947643        Grapes  Jan 1
 175      3302       482   0.936777   2.459543       Apricot  Jan 1
 176      4240       116   1.016865  19.376489   Blueberries  Jan 1
 
 [177 rows x 6 columns],
      prodcode  quantity  price_usd  weight_lb       product   date
 0        4134       498   2.153334  17.287309  Apple (Gala)  Jan 1
 1        4012       

### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [19]:
#Jonnas' assitance, in order to iterate through all functions
[process_data(file, 'Chicago') for file in files]

[     prodcode  quantity  price_usd  weight_lb       product   date
 0        4081       334   1.921072   7.568684      Eggplant  Jan 1
 1        3302       237   1.494549  12.068491       Apricot  Jan 1
 2        4131       426   0.735211   2.926420  Apple (Fuji)  Jan 1
 3        4131       213   3.045255  18.074397  Apple (Fuji)  Jan 1
 4        4134       439   1.158079  10.223446  Apple (Gala)  Jan 1
 ..        ...       ...        ...        ...           ...    ...
 134      4404       306   2.607476  20.696594         Peach  Jan 1
 135      4012       456   1.270081  21.778590        Orange  Jan 1
 136      4062       382   1.309541   8.338494      Cucumber  Jan 1
 137      4796       269   1.098525   4.915191        Tomato  Jan 1
 138      4081       220   3.143239  15.287519      Eggplant  Jan 1
 
 [139 rows x 6 columns],
      prodcode  quantity  price_usd  weight_lb   product   date
 0        4062       195   1.709112  10.068821  Cucumber  Jan 2
 1        4159       220   2.

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [20]:
#concatination of lists into a nascent dataframe
data_list=[process_data(file, 'Chicago') for file in files]

In [21]:
data_df= pd.concat(data_list)

In [22]:
type(data_df)

pandas.core.frame.DataFrame

In [23]:
type(data_list)

list

In [24]:
type(data_df)

pandas.core.frame.DataFrame

### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [25]:
#Jonna' collaboration;
city_dict = [
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
    ]

SyntaxError: invalid syntax (<ipython-input-25-501f7ad8d223>, line 3)

In [26]:
files = ['data/Jan 1.xlsx', 'data/Jan 2.xlsx', 'data/Jan 3.xlsx','data/Jan 4.xlsx','data/Jan 5.xlsx','data/Jan 6.xlsx','data/Jan 7.xlsx','data/Jan 8.xlsx','data/Jan 9.xlsx','data/Jan 10.xlsx', 'data/Jan 11.xlsx', 'data/Jan 12.xlsx','data/Jan 13.xlsx', 'data/Jan 14.xlsx', 'data/Jan 15.xlsx' , 'data/Jan 17.xlsx', 'data/Jan 18.xlsx', 'data/Jan 19.xlsx', 'data/Jan 20.xlsx', 'data/Jan 21.xlsx','data/Jan 22.xlsx','data/Jan 23.xlsx','data/Jan 24.xlsx','data/Jan 25.xlsx','data/Jan 26.xlsx','data/Jan 27.xlsx','data/Jan 28.xlsx','data/Jan 29.xlsx','data/Jan 30.xlsx','data/Jan 31.xlsx']

In [27]:
for q, e in city_dict.items():
    # Create list comprehension to combine all files for city
    data_list = [process_data(file, q) for file in files]
    #Create dataframe from list
    data_df = pd.concat(data_list)
#     # Add city column
    data_df['city'] = q
#     # Rearrange columns
#     data_df = data_df.loc[:, [‘city’, ‘date’, ‘product’, ‘prodcode’, ‘quantity’, ‘weight_lb’, ‘price_usd’]]
#     data_df.to_csv(‘output/’+e)
print(data_df)

NameError: name 'city_dict' is not defined

In [28]:
type(data_df)
print(data_df)

     prodcode  quantity  price_usd  weight_lb       product    date
0        4081       334   1.921072   7.568684      Eggplant   Jan 1
1        3302       237   1.494549  12.068491       Apricot   Jan 1
2        4131       426   0.735211   2.926420  Apple (Fuji)   Jan 1
3        4131       213   3.045255  18.074397  Apple (Fuji)   Jan 1
4        4134       439   1.158079  10.223446  Apple (Gala)   Jan 1
..        ...       ...        ...        ...           ...     ...
164      4011       440   3.240441  14.530989        Banana  Jan 31
165      4011       199   2.946286  16.260957        Banana  Jan 31
166      4309       136   1.587484  18.446122        Lychee  Jan 31
167      4412       102   3.060141   7.197802          Pear  Jan 31
168      4134       321   1.682247  21.666202  Apple (Gala)  Jan 31

[4502 rows x 6 columns]


In [29]:
type(data_list)

list

In [30]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run
#Iteration through each city

for cityname, cityfilename in city_dict.items():
        
        #Establish a list of dataframes for each city 
        city_dfs = [process_data(filename, cityname) for filename in files]
        
        #Merge each DF into one DF
        metros_df = pd.concat(city_dfs)
        
        #Include a column for each city
        metros_df['city'] = cityname
        
        #Order column nomenclature
        metros_df = metros_df.loc[:, {'city', ' date','price_usd', 'product', 'prodcode', 'quantity', 'weight_lbs'}]
        
        #Save DF to a csv file
        metros_df.to_csv('output/' + cityfilename)
        
        print(f'{cityname} ready')
        

NameError: name 'city_dict' is not defined

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [ ]:
master_csv_list = [‘atl.csv’, ‘atx.csv’, ‘bos.csv’, ‘chi.csv’, ‘den.csv’, ‘lax.csv’, ‘nyc.csv’, ‘sf.csv’,
‘sea.csv’, ‘dc.csv’]

#establish a nascent lost for all dataframes
master_df_list = []

for q in master_csv_list:
    master_list.append(pd.read_csv(‘output/’+q))
    
#combine dataframes with concatenation 
master_df = pd.concat(master_df_list)

# Allocated and categorised by state in order to find averages
master_df.groupby(‘city’)[[‘quantity’, ‘weight_lb’, ‘price_us’]].mean()

# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).